In [1]:
import cv2
import numpy as np


## grabs frame from camera one or two
def grabCam(camNum):
    ## cam indexes for camera number
    cam1 = 0
    cam2 = 1
    if(camNum==1):
        cam = cv2.VideoCapture(cam1)
        ret, frame = cam.read()    
        print(ret)
        cam.release()
        return frame
    if(camNum==2):
        cam = cv2.VideoCapture(cam2)
        ret, frame = cam.read()    
        print(ret)
        cam.release()
        return frame
 
## shows a frame
def showPic(fram):
    ## Show frame
    cv2.imshow("image",fram)
    k= cv2.waitKey(0)
    if k==27:
        cv2.destroyAllWindows()
    ## Deactivate Program
    cv2.destroyAllWindows()

## finds circle with color mask
def findCircle(frame):
    ## color mask
    lower = np.array([90,40,200])
    upper = np.array([140,100,255])
    frame = cv2.inRange(cv2.cvtColor(frame,cv2.COLOR_BGR2HSV),lower,upper)

    ## contour detection
    contours, _ = cv2.findContours(frame,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        if cv2.contourArea(contour) >=1000:
            x,y,w,h = cv2.boundingRect(contour)
            if x<1100:
              return [x,y,w,h]
    return -1
## finds square with color mask
def findSquare(frame):
    ## color mask
    lower = np.array([0,80,175])
    upper = np.array([15,200,255])
    frame = cv2.inRange(cv2.cvtColor(frame,cv2.COLOR_BGR2HSV),lower,upper)
    
    ## contour detection
    contours, _ = cv2.findContours(frame,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        if cv2.contourArea(contour) >=1000:
            x,y,w,h = cv2.boundingRect(contour)
            if x<1100:
                return [x,y,w,h]
    return -1

## finds triangle cross and star with single color mask
def findTriangleCrossStar(frame):
    ## initialize star and cross to -1 
    star = -1
    cross = -1
    maybeCross = -1
    maybeStar = -1
    triangle = -1 
    ## color mask
    lower = np.array([15,10,220])
    upper = np.array([60,150,255])
    frame = cv2.inRange(cv2.cvtColor(frame,cv2.COLOR_BGR2HSV),lower,upper)
    ## contour detection
    
    contours, _ = cv2.findContours(frame,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        if cv2.contourArea(contour) >=1000:
            x,y,w,h = cv2.boundingRect(contour)
            if x<1100:
                epsilon = 0.02 * cv2.arcLength(contour,True)
                approx = cv2.approxPolyDP(contour,epsilon,True)
                print("1 ",len(approx))
                if len(approx) == 12 or len(approx)==13:
                    cross = [x,y,w,h]
                    continue
                if len(approx) == 11:
                    maybeCross = [x,y,w,h]
                    continue
                epsilon = 0.07 * cv2.arcLength(contour,True)
                approx = cv2.approxPolyDP(contour,epsilon,True)
                print("2 ",len(approx))
                if len(approx) == 5 or len(approx)==6:
                    star = [x,y,w,h]
                    continue
                if len(approx) == 4:
                    maybeStar = [x,y,w,h]
                    continue
                if len(approx) == 3:
                    triangle = [x,y,w,h]
    if cross ==-1:
        cross = maybeCross
    if star==-1 and cross!=maybeCross:
        star = maybeCross
    if star==-1:
        star = maybeStar
    if triangle == -1:
        triangle = maybeStar
    return triangle, cross, star

## returns list with [x,y,w,h] for circle, triangle, square, star, cross
def findBlocks(frame):
    circle = findCircle(frame)
    square = findSquare(frame)
    triangle, cross, star = findTriangleCrossStar(frame)
    return circle,triangle,square,star,cross

def findRobot(frame):
    ## sometimes robot area is smaller or splits into two contours so we will always pick the largest area
    ## initialize 
    robot = [0,0,0,0]
    area = 20
    ## lower and upper color bounds (hue,sat,val)
    ## hue maxes at 179 others 255
    lower = np.array([105,230,0])
    upper = np.array([120,255,255])
    
    frame = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
    frame = cv2.inRange(frame,lower,upper)
    contours, _ = cv2.findContours(frame,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    ## look at each contour
    for contour in contours:
    ## to filter out noise make sure area over a certain ammount - try 800
        if cv2.contourArea(contour) >area:
            ## coordinates of top left contour
            x,y,w,h = cv2.boundingRect(contour)
            if x>400 and x<1500 and w*h>robot[2]*robot[3]:
                print("Robot",x,y,w,h)
                robot = [x,y,w,h]

    print("Final Robot",robot[0],robot[1],robot[2],robot[3])
    return robot

## shape is given by list [x,y,w,h]
def findMiddle(shape):
    x = shape[0]+(int(shape[2]/2))
    y = shape[1]+(int(shape[3]/2))
    return [x,y]

In [3]:
## servos plug in black away from robot
## check port in terminal with 
import serial
import time
import math

## y and z should not be negative

## turns degrees into microseconds for serial command
def angleToMicro(angle):
    return round(angle*11.11111)+500

## Does the inverse kinematics
## returns the microseconds in order
def findServos(curr):
    ##initialize x,y,z
    x = curr[0]
    y = curr[1]
    z = curr[2]
    
    ##
    ## servo 0
    ##
    if x==0:
        degree = 90
    if x>0:
        degree = math.degrees(math.atan(y/x))
    if x<0:
        degree = 180 - math.degrees(math.atan(y/(x*-1)))
    serv0 = angleToMicro(degree)
    
    ##
    ## servo 1 and 2
    ##
    
    ## measurements
    beginArm = 3
    armOne = 6
    armTwo = 7.5
    armThree = 4
    ## find "x"
    v = math.sqrt(x**2+y**2)
    ## find "y"
    z = z+(armThree-beginArm)
    ## find the two possible angles
    servTwo_plus = math.atan2(math.sqrt(1-((v**2+z**2-armOne**2-armTwo**2)/(2*armOne*armTwo))**2),(v**2+z**2-armOne**2-armTwo**2)/(2*armOne*armTwo))
    servTwo_minus = math.atan2(-1*(math.sqrt(1-((v**2+z**2-armOne**2-armTwo**2)/(2*armOne*armTwo))**2)),(v**2+z**2-armOne**2-armTwo**2)/(2*armOne*armTwo))
    if servTwo_plus>0:
        radTwo = servTwo_plus
    if servTwo_minus>0:
        radTwo = servTwo_minus
    k1 = armOne+(armTwo*math.cos(radTwo))
    k2 = armTwo*math.sin(radTwo)
    degreeOne = math.degrees(math.atan2(z,v)+math.atan2(k2,k1))
    degreeTwo = math.degrees(radTwo)

    ##
    ## servo 3
    ##

    
    serv3 = 180 - (degreeOne + (180-degreeTwo))
    serv3 = angleToMicro(serv3) 
    serv1 = angleToMicro(degreeOne)
    serv2 = angleToMicro(degreeTwo)
    

    return serv0,serv1,serv2,serv3

## cleans up findServos and returns command line not servos
def getSerialCommand(curr,ser4):
    ser0,ser1,ser2,ser3 = findServos(curr)
    if ser4 ==-1:
        ser4 = ser0
    return f"#0 P{ser0} T1500 #3 P{ser3} T1500 #1 P{ser1} T1500 #2 P{ser2} T1500 #4 P{ser4} T1500 \r"
 
## sends serial
def sendSerial(commandString):
    port = "/dev/tty.usbserial-0001"
    baud = 38400
    try:
        ser = serial.Serial(port,baud,timeout=3)
    except:
        print("error opening serial")
    ser.write(commandString.encode('UTF-8'))
    time.sleep(2)
    ser.close()
    

## unit in inches, curr is list [x,y,z] , serv4 is optional do -1 to ignore
def moveUnitDirect(unit,direction,curr,serv4):
    if direction =="x":
        curr = [curr[0]+unit,curr[1],curr[2]]
        command = getSerialCommand(curr,serv4)
        sendSerial(command)
    elif direction =="y":
        curr = [curr[0],curr[1]+ unit,curr[2]]
        command = getSerialCommand(curr,serv4)
        sendSerial(command)
    elif direction =="z":
        curr = [curr[0],curr[1],curr[2]+unit]
        command = getSerialCommand(curr,serv4)
        sendSerial(command)
    return curr

In [21]:
import time
def grabBlock(blockMid,ajustY,ajustX):
    print(ajustY)
    print(ajustX)
    ## move robot to start pos
    position = [0,7.5,5]
    command = getSerialCommand(position,-1)
    sendSerial(command)
    #####
    ##### Considering the y direction
    #####

    ## find robot cam position
    frame = grabCam(1)
    robot = findRobot(frame)

    ## move one unit 
    position = moveUnitDirect(1,"y",position,-1)

    ## find the robot new position
    frame = grabCam(1)
    newRobot = findRobot(frame)

    ## distance moved by 1 unit
    dist = robot[0]-newRobot[0]
    

    ## find new unit to reach shape
    unit = (newRobot[0]-blockMid[0]+ajustY)/dist

    ## set new robot mid to current
    robot = newRobot

    while unit>0.025  or unit<-0.025:
        unit = unit
        ## going to cap at 3 units at a time to prevent as much error as possible
        if unit >=3:
            unit = 3
        if unit<=-3:
            unit = -3 
    
        ## move unit
        position = moveUnitDirect(unit,"y",position,-1)
    
        ## find the robot new position
        frame = grabCam(1)
        newRobot = findRobot(frame)
    
    
        ## distance moved per one unit averaged with new and old dist value
        dist = (robot[0]-newRobot[0])/unit

        ## find new unit to reach shape
        if dist ==0:
            unit = 0
        else:
            unit = (newRobot[0]-blockMid[0]+ajustY)/dist
    print("DONE")

    #####
    ##### Considering the x direction
    #####

    ## find robot cam position
    frame = grabCam(1)
    robot = findRobot(frame)

    ## get center points
    robotMid = findMiddle(robot)

    ## move one unit 
    position = moveUnitDirect(1,"x",position,-1)

    ## find the robot new position
    frame = grabCam(1)

    ## get center points
    newRobotMid = findMiddle(findRobot(frame))

    ## distance moved by 1 unit
    dist = robotMid[1]-newRobotMid[1]

    ## find new unit to reach shape
    unit = (newRobotMid[1]-blockMid[1]+ajustX)/dist

    ## set new robot mid to current
    robotMid = newRobotMid

    while unit>0.025 or unit<-0.025: 
        ## move unit
        position = moveUnitDirect(unit,"x",position,-1)
    
        ## find the robot new position
        frame = grabCam(1)
        newRobot = findRobot(frame)
        if newRobot == [0,0,0,0]:
            break
        newRobotMid = findMiddle(newRobot)
        ## distance moved per one unit averaged with new and old dist value
        dist = (robotMid[1]- newRobotMid[1])/unit

        ## find new unit to reach shape
        if dist == 0:
            unit = 0
        else:
            unit = (newRobotMid[1]-blockMid[1]+ajustX)/dist
    print("DONE")

    ## open gripper
    command = "#5 P1500 T500 \r"
    sendSerial(command)

    ## go down to block height
    position = moveUnitDirect(-3.5,"z",position,-1)

    ## close gripper
    command = "#5 P500 T500 \r"
    sendSerial(command)

    return position


#####
#####  CIRCLE
#####


def grabCircle(circle):
    ## calculate X adjustment
    circMid = findMiddle(circle)
    print(circMid)
    ajustX = (-0.17*circMid[1])+120
    if circMid[1]>700:
        ajustX=ajustX-10
    if circMid[1]<400:
        ajustX=ajustX+154
    elif circMid[1]<500:
        ajustX=ajustX+10
    ## calculate Y ajustment
    if circMid[0]>1000:
        ajustY = 30
    else:
        ajustY = 50
    ## grab circle
    return grabBlock(circMid,ajustY,ajustX)


def placeCircle(position):
    ## go up to travel height
    position = moveUnitDirect(3.5,"z",position,-1)

    ## put in circle hole
    position = [0,7.5,8]
    command = getSerialCommand(position,-1)
    sendSerial(command)

    position = [-7,1.5,7.5]
    command = getSerialCommand(position,-1)
    sendSerial(command)

    position = [-7.5,2.25,7.5]
    command = getSerialCommand(position,-1)
    sendSerial(command)


    ## open gripper
    command = "#5 P1500 T1000 \r"
    sendSerial(command)

    ## return
    position = [-7.25,1.25,7.5]
    command = getSerialCommand(position,-1)
    sendSerial(command)
    
    position = [0,7.5,7]
    command = getSerialCommand(position,-1)
    sendSerial(command)

    return position

#####
#####  SQUARE
#####


def grabSquare(square):
    ## calculate X adjustment
    squareMid = findMiddle(square)
    print(squareMid)
    ajustX = (-0.17*squareMid[1])+120
    if squareMid[1]>650:
        ajustX=ajustX-15
    elif squareMid[1]>500:
        ajustX = ajustX-5
    if squareMid[1]<420:
        ajustX=ajustX+15
    ## calculate Y ajustment
    if squareMid[0]>1100:
        ajustY = 0
    else:
        ajustY = 35
    ## grab circle
    return grabBlock(squareMid,ajustY,ajustX)

def placeSquare(position):
    ## go up to travel height
    position = moveUnitDirect(3.5,"z",position,-1)

    ## put in square hole
    position = [0,7.5,8]
    command = getSerialCommand(position,-1)
    sendSerial(command)

    position = [-7,0,8]
    command = getSerialCommand(position,2300)
    sendSerial(command)
    
    position = [-7,0,6.5]
    command = getSerialCommand(position,2300)
    sendSerial(command)
       
    position = [-7.5,0,7]
    command = getSerialCommand(position,2250)
    sendSerial(command)
        
    ## open gripper
    command = "#3 P500 T1000 \r"
    command = "#5 P1500 T1000 \r"
    sendSerial(command)
        
    ## return
    position = [-7.25,0,7.5]
    command = getSerialCommand(position,2250)
    sendSerial(command)
            
    position = [0,7.5,7]
    command = getSerialCommand(position,-1)
    sendSerial(command)

#####
#####  Triangle
#####


def grabTriangle(triangle):
    ## calculate X adjustment
    triangleMid = findMiddle(triangle)
    print(triangleMid)
    ajustX = (-0.17*triangleMid[1])+125
    if triangleMid[1]>650:
        ajustX=ajustX-10
    elif triangleMid[1]>550:
        ajustX=ajustX-5
    if triangleMid[1]<500:
        ajustX=ajustX+10
    ## calculate Y ajustment
    if triangleMid[0]>1100:
        ajustY = 0
    else:
        ajustY = 20
    ## grab circle
    return grabBlock(triangleMid,ajustY,ajustX)

def placeTriangle(position):
    ## go up to travel height
    position = moveUnitDirect(3.5,"z",position,-1)

    ## put in triangle hole
    position = [0,7.5,8]
    command = getSerialCommand(position,-1)
    sendSerial(command)

    position = [-7,3.5,7.5]
    command = getSerialCommand(position,1700)
    sendSerial(command)
    
    position = [-6.75,2.8,6.5]
    command = getSerialCommand(position,1650)
    sendSerial(command)

    command = "#5 P1500 T1000 \r"
    sendSerial(command)
    
    position = [-7,3.5,7.5]
    command = getSerialCommand(position,1700)
    sendSerial(command)

    position = [0,7.5,7]
    command = getSerialCommand(position,-1)
    sendSerial(command)

####
#### cross
####



## similarto triangle with adjustments
def grabCross(cross):
    ## calculate X adjustment
    crossMid = findMiddle(cross)
    print(crossMid)
    ajustX = (-0.17*crossMid[1])+125
    if crossMid[1]>650:
        ajustX=ajustX-10
    elif crossMid[1]>550:
        ajustX=ajustX-5
    if crossMid[1]<450:
        ajustX=ajustX+10
    ## calculate Y ajustment
    if crossMid[0]>1100:
        ajustY = 10
    else:
        ajustY = 45
    ## grab circle
    return grabBlock(crossMid,ajustY,ajustX)

def placeCross(position):
    ## go up to travel height
    position = moveUnitDirect(3.5,"z",position,-1)

    ## put in cross hole
    position = [0,7.5,8]
    command = getSerialCommand(position,-1)
    sendSerial(command)

    position = [-6,1,8]
    command = getSerialCommand(position,2030)
    sendSerial(command)
    
    position = [-6,1,7]
    command = getSerialCommand(position,2030)
    sendSerial(command)
    
    position = [-5.75,0.75,6.25]
    command = getSerialCommand(position,2100)
    sendSerial(command)
    
    command = "#5 P1500 T1000 \r"
    sendSerial(command)
    
    position = [-5.5,0.5,7.5]
    command = getSerialCommand(position,2000)
    sendSerial(command)


####
#### star
####

## similar to triangle with adjustments
def grabStar(star):
    ## calculate X adjustment
    starMid = findMiddle(star)
    print(starMid)
    ajustX = (-0.17*starMid[1])+120
    if starMid[1]>650:
        ajustX=ajustX-10
    elif starMid[1]>600:
        ajustX=ajustX-5
    if starMid[1]<450:
        ajustX=ajustX+10
    ## calculate Y ajustment
    if starMid[0]>1100:
        ajustY = 5
    else:
        ajustY = 45
    ## grab star
    return grabBlock(starMid,ajustY,ajustX)

def placeStar(position):
    ## go up to travel height
    position = moveUnitDirect(3.5,"z",position,-1)
    
    ## put in cross hole
    position = [0,7.5,8]
    command = getSerialCommand(position,-1)
    sendSerial(command)
    
    position = [-5.5,3,8]
    command = getSerialCommand(position,2030)
    sendSerial(command)
    
    position = [-5.5,3,6.5]
    command = getSerialCommand(position,2200)
    sendSerial(command)
    
    command = "#5 P1500 T1000 \r"
    sendSerial(command)
    
    position = [-5.5,3,7.5]
    command = getSerialCommand(position,2000)
    sendSerial(command)

In [29]:
time.sleep(2)
## start robot but get out of way to find blocks
position = [0,7,5,5]
command = getSerialCommand(position,-1)
sendSerial(command)
position = [7,1,5]
command = getSerialCommand(position,-1)
sendSerial(command)

## get top cam
frame = grabCam(1)

## find the shapes
circle,triangle,square,star,cross = findBlocks(frame)
while cross != -1:
    print("in cross loop")
    position = grabCross(cross)   
    position = placeCross(position)

    position = [0,7,5,5]
    command = getSerialCommand(position,-1)
    sendSerial(command)
    position = [7,1,5]
    command = getSerialCommand(position,-1)
    sendSerial(command)
    
    ## get top cam
    frame = grabCam(1)
    ## find the shapes
    circle,triangle,square,star,cross = findBlocks(frame)

while square!=-1:
    position = grabSquare(square)
    position = placeSquare(position)

    position = [0,7,5,5]
    command = getSerialCommand(position,-1)
    sendSerial(command)
    position = [7,1,5]
    command = getSerialCommand(position,-1)
    sendSerial(command)

    ## get top cam
    frame = grabCam(1)

    ## find the shapes
    square = findSquare(frame)

while circle != -1:
    position = grabCircle(circle)   
    position = placeCircle(position)

    position = [0,7,5,5]
    command = getSerialCommand(position,-1)
    sendSerial(command)
    position = [7,1,5]
    command = getSerialCommand(position,-1)
    sendSerial(command)

    ## get top cam
    frame = grabCam(1)

    ## find the shapes
    circle = findCircle(frame)
while triangle!= -1:
    position = grabTriangle(triangle)
    position = placeTriangle(position)

    position = [0,7,5,5]
    command = getSerialCommand(position,-1)
    sendSerial(command)
    position = [7,1,5]
    command = getSerialCommand(position,-1)
    sendSerial(command)

    ## get top cam
    frame = grabCam(1)
    triangle,cross,star = findTriangleCrossStar(frame)

while star != -1:
    position = grabStar(star)   
    position = placeStar(position)

    position = [0,7,5,5]
    command = getSerialCommand(position,-1)
    sendSerial(command)
    position = [7,1,5]
    command = getSerialCommand(position,-1)
    sendSerial(command)
    
    ## get top cam
    frame = grabCam(1)
    ## find the shapes
    circle,triangle,square,star,cross = findBlocks(frame)

True
1  12
in cross loop
[981, 544]
45
32.519999999999996
True
Robot 1373 625 13 8
Robot 1028 452 34 52
Final Robot 1028 452 34 52
True
Robot 1374 626 13 7
Robot 974 461 22 33
Final Robot 974 461 22 33
True
Robot 1374 626 12 6
Robot 933 460 22 33
Final Robot 933 460 22 33
True
Robot 1374 626 12 6
Robot 935 461 22 33
Final Robot 935 461 22 33
DONE
True
Robot 1374 626 13 7
Robot 935 460 22 36
Final Robot 935 460 22 36
True
Robot 1374 625 13 8
Robot 934 505 22 32
Final Robot 934 505 22 32
True
Robot 1374 626 12 6
Robot 935 503 20 34
Final Robot 935 503 20 34
True
Robot 1373 625 14 7
Robot 934 407 33 37
Final Robot 934 407 33 37
True
Robot 1374 625 13 8
Robot 934 487 21 35
Final Robot 934 487 21 35
True
Robot 1374 626 12 6
Robot 935 459 22 34
Final Robot 935 459 22 34
True
Robot 1374 625 13 8
Robot 935 477 22 35
Final Robot 935 477 22 35
True
Robot 1374 626 12 7
Robot 935 470 22 38
Final Robot 935 470 22 38
True
Robot 1374 625 13 8
Robot 935 472 22 37
Final Robot 935 472 22 37
True
Robot 1

In [10]:
position = [0,7,3]
command = getSerialCommand(position,-1)
sendSerial(command)

In [9]:
command = "#5 P500 T1000 \r"
sendSerial(command)

In [145]:
position = [-7,3.5,7.5]
command = getSerialCommand(position,1650)
sendSerial(command)
position = [-7,3,6.5]
command = getSerialCommand(position,1650)
sendSerial(command)



"""   
command = "#4 P600 T1000 \r"
    sendSerial(command)
    
    ## open gripper
    command = "#5 P1500 T1000 \r"
    sendSerial(command)
    
    ## return
    position = [-7.25,1.25,7.5]
    command = getSerialCommand(position,500)
    sendSerial(command)
        
    position = [0,7.5,7]
    command = getSerialCommand(position,-1)
    sendSerial(command)
    """

'   \ncommand = "#4 P600 T1000 \r"\n    sendSerial(command)\n    \n    ## open gripper\n    command = "#5 P1500 T1000 \r"\n    sendSerial(command)\n    \n    ## return\n    position = [-7.25,1.25,7.5]\n    command = getSerialCommand(position,500)\n    sendSerial(command)\n        \n    position = [0,7.5,7]\n    command = getSerialCommand(position,-1)\n    sendSerial(command)\n    '

In [11]:
## open gripper
command = "#5 P1500 T1000 \r"
sendSerial(command)

In [13]:
position = [-7,3.5,7.5]
command = getSerialCommand(position,1700)
sendSerial(command)

In [163]:
position = [-6.75,2.8,6.5]
command = getSerialCommand(position,1650)
sendSerial(command)

In [104]:
time.sleep(2)
## start robot but get out of way to find blocks
position = [0,7,5,5]
command = getSerialCommand(position,-1)
sendSerial(command)
position = [7,1,5]
command = getSerialCommand(position,-1)
sendSerial(command)

## get top cam
frame = grabCam(1)

## find the shapes
circle,triangle,square,star,cross = findBlocks(frame)

position = grabSquare(square)

True
[1030, 675]
35
-9.750000000000014
True
Robot 1021 465 22 46
Final Robot 1021 465 22 46
True
Robot 966 470 22 38
Final Robot 966 470 22 38
True
Robot 993 472 21 37
Final Robot 993 472 21 37
True
Robot 996 466 27 48
Final Robot 996 466 27 48
DONE
True
Robot 995 470 27 44
Final Robot 995 470 27 44
True
Robot 1026 523 15 16
Robot 996 511 32 41
Final Robot 996 511 32 41
True
Robot 1024 721 13 19
Final Robot 1024 721 13 19
True
Robot 1027 688 16 20
Final Robot 1027 688 16 20
True
Robot 1026 689 16 18
Final Robot 1026 689 16 18
DONE


In [45]:
position = [-7,.5,7.5]
command = getSerialCommand(position,500)
sendSerial(command)
   
position = [-6.75,.5,6.25]
command = getSerialCommand(position,500)
sendSerial(command)

    
command = "#4 P600 T1000 \r"
sendSerial(command)
    
    ## open gripper
command = "#5 P1500 T1000 \r"
sendSerial(command)
    
    ## return
position = [-7.25,1.25,7.5]
command = getSerialCommand(position,500)
sendSerial(command)
        
position = [0,7.5,7]
command = getSerialCommand(position,-1)
sendSerial(command)
    

In [135]:


position = [-7,0,7.5]
command = getSerialCommand(position,2300)
sendSerial(command)

In [137]:
position = [-7.5,0.5,7]
command = getSerialCommand(position,2250)
sendSerial(command)

In [139]:
    ## open gripper
command = "#5 P1500 T1000 \r"
sendSerial(command)

In [141]:
  ## return
position = [-7.25,0,7.5]
command = getSerialCommand(position,2250)
sendSerial(command)

In [133]:
 ## go up to travel height
position = moveUnitDirect(3.5,"z",position,-1)

## put in square hole
position = [0,7.5,8]
command = getSerialCommand(position,-1)
sendSerial(command)

   

In [63]:

## put in triangle hole
  
position = [0,7.5,8]
command = getSerialCommand(position,-1)
sendSerial(command)

In [65]:
position = [-6,1,8]
command = getSerialCommand(position,2030)
sendSerial(command)

In [67]:
position = [-6,1,7]
command = getSerialCommand(position,2030)
sendSerial(command)

In [83]:
position = [-5.75,0.75,6.25]
command = getSerialCommand(position,2100)
sendSerial(command)

In [85]:
    ## open gripper
command = "#5 P1500 T1000 \r"
sendSerial(command)

In [87]:
position = [-5.5,0.5,7.5]
command = getSerialCommand(position,2000)
sendSerial(command)

'\ncommand = "#5 P1500 T1000 \r"\nsendSerial(command)\n    \nposition = [-5.5,0.5,7.5]\ncommand = getSerialCommand(position,2000)\nsendSerial(command)\n'

In [139]:
## put in cross hole


position = [-6,1,8]
command = getSerialCommand(position,2030)
sendSerial(command)
    
position = [-6,1,7]
command = getSerialCommand(position,2030)
sendSerial(command)

position = [-5.75,0.75,6.25]
command = getSerialCommand(position,2100)
sendSerial(command)

command = "#5 P1500 T1000 \r"
sendSerial(command)

position = [-5.5,0.5,7.5]
command = getSerialCommand(position,2000)
sendSerial(command)

In [143]:
frame = grabCam(1)
triangle,cross,star = findTriangleCrossStar(frame) 
if triangle ==-1:
    triangle,cross,star = findTriangleCrossStar(frame) 
while triangle!= -1:
    position = grabTriangle(triangle)
    position = placeTriangle(position)

    position = [0,7,5,5]
    command = getSerialCommand(position,-1)
    sendSerial(command)
    position = [7,1,5]
    command = getSerialCommand(position,-1)
    sendSerial(command)

    ## get top cam
    frame = grabCam(1)
    triangle,cross,star = findTriangleCrossStar(frame)


True
1  5
2  3
[931, 615]
40
-2.5500000000000114
True
Robot 1003 461 19 33
Final Robot 1003 461 19 33
True
Robot 946 460 19 33
Final Robot 946 460 19 33
True
Robot 951 479 7 10
Robot 894 467 18 23
Final Robot 894 467 18 23
True
Robot 949 478 7 10
Robot 891 469 19 21
Final Robot 891 469 19 21
DONE
True
Robot 949 478 6 10
Robot 891 466 20 24
Final Robot 891 466 20 24
True
Robot 947 520 8 10
Robot 890 514 18 24
Final Robot 890 514 18 24
True
Robot 893 616 15 22
Final Robot 893 616 15 22
True
Robot 896 614 15 23
Final Robot 896 614 15 23
DONE
True


In [63]:
time.sleep(2)
## start robot but get out of way to find blocks
position = [0,7,5,5]
command = getSerialCommand(position,-1)
sendSerial(command)
position = [7,1,5]
command = getSerialCommand(position,-1)
sendSerial(command)

## get top cam
frame = grabCam(1)

## find the shapes
circle,triangle,square,star,cross = findBlocks(frame)
print("star",star)
while star != -1:
    position = grabStar(star)   
    """position = placeStar(position)

    position = [0,7,5,5]
    command = getSerialCommand(position,-1)
    sendSerial(command)
    position = [7,1,5]
    command = getSerialCommand(position,-1)
    sendSerial(command)
    
    ## get top cam
    frame = grabCam(1)
    ## find the shapes
    circle,triangle,square,star,cross = findBlocks(frame)
    """
    star = -1



True
1  8
2  5
star [947, 518, 66, 64]
[980, 550]
45
36.5
True
Robot 986 443 21 38
Final Robot 986 443 21 38
True
Robot 931 443 21 37
Final Robot 931 443 21 37
True
Robot 931 445 21 35
Final Robot 931 445 21 35
DONE
True
Robot 931 444 21 37
Final Robot 931 444 21 37
True
Robot 934 488 21 34
Final Robot 934 488 21 34
True
Robot 933 502 22 29
Final Robot 933 502 22 29
True
Robot 933 497 22 34
Final Robot 933 497 22 34
DONE


In [75]:

## go up to travel height
position = moveUnitDirect(3.5,"z",position,-1)

## put in cross hole
position = [0,7.5,8]
command = getSerialCommand(position,-1)
sendSerial(command)

position = [-5.5,3,8]
command = getSerialCommand(position,2050)
sendSerial(command)
    
position = [-5.5,3,6.5]
command = getSerialCommand(position,2200)
sendSerial(command)

command = "#5 P1500 T1000 \r"
sendSerial(command)

position = [-5.5,3,7.5]
command = getSerialCommand(position,2000)
sendSerial(command)


In [106]:

## go up to travel height
position = moveUnitDirect(3.5,"z",position,-1)

## put in square hole
position = [0,7.5,8]
command = getSerialCommand(position,-1)
sendSerial(command)

position = [-7,0,8]
command = getSerialCommand(position,2300)
sendSerial(command)

position = [-7.5,0.5,7]
command = getSerialCommand(position,2300)
sendSerial(command)
 
position = [-7.5,0,6.5]
command = getSerialCommand(position,2250)
sendSerial(command)
    
## open gripper
command = "#5 P1500 T1000 \r"
sendSerial(command)
    
## return
position = [-7.25,0,7.5]
command = getSerialCommand(position,2250)
sendSerial(command)
        
position = [0,7.5,7]
command = getSerialCommand(position,-1)
sendSerial(command)
